In [ ]:
#imports
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

In [4]:
parking = pd.read_csv('treas_parking_payments_2017_datasd.csv')

In [5]:
# 1.) remove unnecessary columns: all we need is pole_id, time_start and meter_expire
parking = parking.drop(['uuid'], axis=1)
parking = parking.drop(['trans_amt'], axis=1)
parking = parking.drop(['pay_method'], axis=1)
parking = parking.drop(['meter_type'], axis=1)
parking = parking.drop(['meter_expire'], axis=1)

In [6]:
#clean the data, removing any na rows 
parking.dropna(how='any')

,pole_id,trans_start
0,SL-216,2017-01-01 00:15:18
1,5-402,2017-01-01 01:03:49
2,G-503,2017-01-01 01:23:53
3,G-503,2017-01-01 01:27:01
4,G-503,2017-01-01 01:27:19
5,G-503,2017-01-01 01:27:35
6,G-503,2017-01-01 01:27:43
7,3-3941,2017-01-01 01:29:56
8,3-3941,2017-01-01 01:30:09
9,4-4065,2017-01-01 03:01:02


In [7]:
# 2.) using pole_id we can connect with lat/long. We will be grouping by pole_id to create
#     a table with the following columns:
#     (pole_id, latitude, longitude, count_am_early, count_am_peak, 
#        count_midday, count_pm_early, count_pm_late, count_daily)
# NOTE: we will need to decide whether to use raw numbers or averages of counts per section per day

In [8]:
park_loc = pd.read_csv('treas_parking_meters_loc_datasd.csv')

In [9]:
park_loc = park_loc[['pole', 'longitude', 'latitude']]
park_loc.columns = ['pole_id', 'longitude', 'latitude']
park_loc = park_loc.dropna(how='any')

In [10]:
park_df = parking.merge(park_loc, how='left')
park_df = park_df.dropna(how='any')
park_df

,pole_id,trans_start,longitude,latitude
0,SL-216,2017-01-01 00:15:18,-117.162112,32.710495
1,5-402,2017-01-01 01:03:49,-117.160212,32.709683
2,G-503,2017-01-01 01:23:53,-117.159897,32.712561
3,G-503,2017-01-01 01:27:01,-117.159897,32.712561
4,G-503,2017-01-01 01:27:19,-117.159897,32.712561
5,G-503,2017-01-01 01:27:35,-117.159897,32.712561
6,G-503,2017-01-01 01:27:43,-117.159897,32.712561
7,3-3941,2017-01-01 01:29:56,-117.162679,32.749437
8,3-3941,2017-01-01 01:30:09,-117.162679,32.749437
9,4-4065,2017-01-01 03:01:02,-117.161294,32.753444


In [11]:
park_counts = park_df.groupby(['pole_id']).agg('count')
park_counts = park_counts['trans_start'] #remove unnecessary columns
park_counts = park_counts.to_frame()
park_counts['pole_id'] = park_counts.index
park_counts = park_counts.merge(park_loc, how='left')
park_counts.columns = ['total_count', 'pole_id', 'longitude', 'latitude']
park_counts

,total_count,pole_id,longitude,latitude
0,938,1-1004,-117.163929,32.715904
1,983,1-1006,-117.163930,32.716037
2,952,1-1008,-117.163931,32.716169
3,875,1-1020,-117.161278,32.717890
4,1077,1-1310,-117.163951,32.719024
5,1108,1-1312,-117.163952,32.719161
6,1070,1-1313,-117.163770,32.719298
7,1161,1-1314,-117.163953,32.719453
8,968,1-1315,-117.163772,32.719571
9,976,1-1317,-117.163772,32.719707


In [12]:
# we need to now find the number of days in the transactions dataset
# we will be using this in order to get the count of transactions PER DAY
park_df['trans_start'] = pd.to_datetime(park_df['trans_start'])
dates = park_df['trans_start']
am_early_d = {}
am_peak_d = {}
midday_d = {}
pm_peak_d = {}
pm_late_d = {}

def classify(x): 
    hour = x.time().hour
    if hour <=6:
        return 'am_early'
    elif hour <=9:
        return 'am_peak'
    elif hour <=14:
        return 'midday'
    elif hour <=19:
        return 'pm_peak'
    else:
        return 'pm_late'
park_df['time_slot'] = park_df['trans_start'].apply(classify)
park_df

,pole_id,trans_start,longitude,latitude,time_slot
0,SL-216,2017-01-01 00:15:18,-117.162112,32.710495,am_early
1,5-402,2017-01-01 01:03:49,-117.160212,32.709683,am_early
2,G-503,2017-01-01 01:23:53,-117.159897,32.712561,am_early
3,G-503,2017-01-01 01:27:01,-117.159897,32.712561,am_early
4,G-503,2017-01-01 01:27:19,-117.159897,32.712561,am_early
5,G-503,2017-01-01 01:27:35,-117.159897,32.712561,am_early
6,G-503,2017-01-01 01:27:43,-117.159897,32.712561,am_early
7,3-3941,2017-01-01 01:29:56,-117.162679,32.749437,am_early
8,3-3941,2017-01-01 01:30:09,-117.162679,32.749437,am_early
9,4-4065,2017-01-01 03:01:02,-117.161294,32.753444,am_early


In [ ]:
def checkSeriesColumn(s, col):
    val = False
    for row in s.keys().to_series().str.contains(col): 
        if(row == True):
            val = True
    return val
def set_temporal_counts(p_id):
    v_counts = park_df.loc[park_df['pole_id'] == p_id]['time_slot'].value_counts(dropna=False)
    park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
    park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
    park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
#     park_counts.loc[park_counts['pole_id'] == ]
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
#      park_counts.loc[park_counts['pole_id'] == p_id,'am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
#     park_counts.loc[park_counts['pole_id'] == p_id,'pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']


park_counts['pole_id'].apply(set_temporal_counts)




# v_counts = park_df.loc[park_df['pole_id'] == 'N-1003']['time_slot'].value_counts(dropna=False)
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_early'] = 0 if not checkSeriesColumn(v_counts, 'am_early') else v_counts['am_early']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','am_peak'] =  0 if not checkSeriesColumn(v_counts, 'am_peak') else v_counts['am_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','midday'] =  0 if not checkSeriesColumn(v_counts, 'midday') else v_counts['midday']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_peak'] =  0 if not checkSeriesColumn(v_counts, 'pm_peak') else v_counts['pm_peak']
# park_counts.loc[park_counts['pole_id'] == 'N-1003','pm_late'] = 0 if not checkSeriesColumn(v_counts, 'pm_late') else v_counts['pm_late']
# park_counts.loc[park_counts['pole_id'] == 'N-1003']



In [117]:

max_date = dates.max()
min_date = dates.min()
days_elapsed = (max_date - min_date).days + 1 #to round off this number
days_elapsed

142

In [115]:

park_counts


,total_count,pole_id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late
0,938,1-1004,-117.163929,32.715904,444.0,26.0,180.0,278.0,10.0
1,983,1-1006,-117.163930,32.716037,486.0,25.0,166.0,301.0,5.0
2,952,1-1008,-117.163931,32.716169,440.0,12.0,197.0,299.0,4.0
3,875,1-1020,-117.161278,32.717890,329.0,14.0,164.0,363.0,5.0
4,1077,1-1310,-117.163951,32.719024,619.0,50.0,298.0,109.0,1.0
5,1108,1-1312,-117.163952,32.719161,672.0,28.0,307.0,98.0,3.0
6,1070,1-1313,-117.163770,32.719298,647.0,27.0,309.0,85.0,2.0
7,1161,1-1314,-117.163953,32.719453,666.0,45.0,331.0,118.0,1.0
8,968,1-1315,-117.163772,32.719571,591.0,13.0,272.0,92.0,0.0
9,976,1-1317,-117.163772,32.719707,606.0,17.0,292.0,61.0,0.0


In [116]:
park_counts.to_csv("Parking_Counts.csv");

In [118]:
# let's now get average counts of parked vehicles
# dividing by total days to give per daily expected counts 
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x / days_elapsed)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x / days_elapsed)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x / days_elapsed)
park_counts['midday'] = park_counts['midday'].apply(lambda x: x / days_elapsed)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x: x / days_elapsed)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x: x / days_elapsed)


,total_count,pole_id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late
0,6.605634,1-1004,-117.163929,32.715904,3.126761,0.183099,1.267606,1.957746,0.070423
1,6.922535,1-1006,-117.163930,32.716037,3.422535,0.176056,1.169014,2.119718,0.035211
2,6.704225,1-1008,-117.163931,32.716169,3.098592,0.084507,1.387324,2.105634,0.028169
3,6.161972,1-1020,-117.161278,32.717890,2.316901,0.098592,1.154930,2.556338,0.035211
4,7.584507,1-1310,-117.163951,32.719024,4.359155,0.352113,2.098592,0.767606,0.007042
5,7.802817,1-1312,-117.163952,32.719161,4.732394,0.197183,2.161972,0.690141,0.021127
6,7.535211,1-1313,-117.163770,32.719298,4.556338,0.190141,2.176056,0.598592,0.014085
7,8.176056,1-1314,-117.163953,32.719453,4.690141,0.316901,2.330986,0.830986,0.007042
8,6.816901,1-1315,-117.163772,32.719571,4.161972,0.091549,1.915493,0.647887,0.000000
9,6.873239,1-1317,-117.163772,32.719707,4.267606,0.119718,2.056338,0.429577,0.000000


In [119]:
# we then multiply this amount by the mean number of people per vehicle
# as per https://www.rita.dot.gov/bts/sites/rita.dot.gov.bts/files/publications/highlights_of_the_2001_national_household_travel_survey/html/table_a15.html
ppl_per_vehicle = 1.63
park_counts['total_count'] = park_counts['total_count'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_early'] = park_counts['am_early'].apply(lambda x: x * ppl_per_vehicle)
park_counts['am_peak'] = park_counts['am_peak'].apply(lambda x: x * ppl_per_vehicle)
park_counts['midday'] = park_counts['midday'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_peak'] = park_counts['pm_peak'].apply(lambda x:  x * ppl_per_vehicle)
park_counts['pm_late'] = park_counts['pm_late'].apply(lambda x:  x * ppl_per_vehicle)
park_counts

,total_count,pole_id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late
0,10.767183,1-1004,-117.163929,32.715904,5.096620,0.298451,2.066197,3.191127,0.114789
1,11.283732,1-1006,-117.163930,32.716037,5.578732,0.286972,1.905493,3.455141,0.057394
2,10.927887,1-1008,-117.163931,32.716169,5.050704,0.137746,2.261338,3.432183,0.045915
3,10.044014,1-1020,-117.161278,32.717890,3.776549,0.160704,1.882535,4.166831,0.057394
4,12.362746,1-1310,-117.163951,32.719024,7.105423,0.573944,3.420704,1.251197,0.011479
5,12.718592,1-1312,-117.163952,32.719161,7.713803,0.321408,3.524014,1.124930,0.034437
6,12.282394,1-1313,-117.163770,32.719298,7.426831,0.309930,3.546972,0.975704,0.022958
7,13.326972,1-1314,-117.163953,32.719453,7.644930,0.516549,3.799507,1.354507,0.011479
8,11.111549,1-1315,-117.163772,32.719571,6.784014,0.149225,3.122254,1.056056,0.000000
9,11.203380,1-1317,-117.163772,32.719707,6.956197,0.195141,3.351831,0.700211,0.000000


In [21]:
park_counts = pd.read_csv("Parking_Counts.csv");
####Rough Calculations for algorithm
long_max = park_counts['longitude'].max()
long_min = park_counts['longitude'].min()
lat_max = park_counts['latitude'].max()
lat_min = park_counts['latitude'].min()
lat_dif = lat_max - lat_min
long_dif = long_max - long_min
NUMBER_BLOCKS_ROOT = 10 #this means 100 blocks 10x10
lat_gap = lat_dif / NUMBER_BLOCKS_ROOT
long_gap = long_dif / NUMBER_BLOCKS_ROOT

def classify_blocks(s):
    park_counts.loc[park_counts['pole_id'] == s,'row'] =  (park_counts.loc[park_counts['pole_id'] == s,'latitude'] - lat_min) // lat_gap
    park_counts.loc[park_counts['pole_id'] == s,'col'] = (park_counts.loc[park_counts['pole_id'] == s,'longitude'] - long_min) // long_gap

park_counts['pole_id'].apply(classify_blocks)

park_counts

,Unnamed: 0,Unnamed: 0.1,total_count,pole_id,longitude,latitude,midday,am_early,am_peak,pm_peak,pm_late,row,col
0,0,0,0.123595,1-1004,-117.163929,32.715904,0.058503,0.003426,0.023718,0.036631,0.001318,6.0,9.0
1,1,1,0.129525,1-1006,-117.163930,32.716037,0.064038,0.003294,0.021873,0.039661,0.000659,6.0,9.0
2,2,2,0.125440,1-1008,-117.163931,32.716169,0.057976,0.001581,0.025958,0.039398,0.000527,6.0,9.0
3,3,3,0.115294,1-1020,-117.161278,32.717890,0.043351,0.001845,0.021609,0.047831,0.000659,6.0,9.0
4,4,4,0.141910,1-1310,-117.163951,32.719024,0.081562,0.006588,0.039266,0.014362,0.000132,6.0,9.0
5,5,5,0.145995,1-1312,-117.163952,32.719161,0.088546,0.003689,0.040452,0.012913,0.000395,6.0,9.0
6,6,6,0.140988,1-1313,-117.163770,32.719298,0.085252,0.003558,0.040715,0.011200,0.000264,6.0,9.0
7,7,7,0.152979,1-1314,-117.163953,32.719453,0.087755,0.005929,0.043614,0.015548,0.000132,6.0,9.0
8,8,8,0.127548,1-1315,-117.163772,32.719571,0.077873,0.001713,0.035840,0.012122,0.000000,6.0,9.0
9,9,9,0.128602,1-1317,-117.163772,32.719707,0.079849,0.002240,0.038475,0.008038,0.000000,6.0,9.0


In [29]:
print(long_max)
print(long_min)
print(long_dif)
print(lat_max)
print(lat_min)
print(lat_dif)
park_counts = park_counts.loc[park_counts['longitude'] != -180.0 ] #remove the outlier
long_min = park_counts['longitude'].min()
long_min

-117.0
-180.0
63.0
33.0
32.1
0.9


-117.25069099999999

In [23]:
park_counts.to_csv("Parking_Counts_grid.csv");